In [1]:
import easyocr
from transformers import pipeline
import cv2
reader=easyocr.Reader(['en'],gpu=False)

Using CPU. Note: This module is much faster with a GPU.


In [2]:
translation_de=pipeline('translation_en_to_de')
translation_de=pipeline('translation_en_to_de')
translation_ur=pipeline("translation", model="Helsinki-NLP/opus-mt-en-ur")

No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\PCR\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\PCR\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
def translation_transformers(text,language):
    if 'ur'==language:
        results=translation_ur(text)
    if 'de'==language:
        results=translation_de(text)
    if 'ja'==language:
        results=translation_ja(text)
    return results[0]['translation_text']

In [6]:
def video_to_text_translation(camera_no_or_video_path=0,want_to_translate=True,
                              language_to_translate='ur',button_to_close_cam='q'):
    cap=cv2.VideoCapture(camera_no)
    w,h=720,480
    cap.set(3, w)
    cap.set(4, h)
    
    top_left=(100,100)
    top_right=(w-100,h-100)
    iterable=0
    ctime=0
    ptime=0
    
    while cap.isOpened():
        success,frame=cap.read()
        if reader.readtext(frame):
            text=' '
            for result in reader.readtext(frame):
                txt=result[1]+' '
                # print(txt)
                texts=text+txt
                text=texts
            if want_to_translate:
                filters="!#$%&*+-/;<=>@^_`|'~\t\n?"
                for filter in filters:
                    filter_text=text.replace(filter,' ')
                    if filter_text!=text:
                        text=filter_text
                        if filter=='  ':
                            filter_text=text.replace(filter,' ')
                    else:
                        filter_text=text
                text=str(text)
                translated_text=translation_transformers(text,language_to_translate)
                return translated_text
            else:
                text=str(text)
                return text
        ctime=time.time()
        fps=1/(ctime-ptime)
        ptime=ctime
        cv2.putText(frame,str(fps),(50,50),cv2.FONT_HERSHEY_PLAIN,2, color=(255,0,0))
        cv2.imshow('text detection',frame)
        if cv2.waitKey(1) & 0XFF==ord(button_to_close_cam):
            break
    cap.release()
    cv2.destroyAllWindows()